# ELECTRIC VEHICLES AVAILABLE FOR SALE IN GREATER MONTREAL 

The goal of the mini project is to show  the implementation of an **ETL process** by gathering the data of different electric vehicles available for sale in Montreal.
1. The extraction will be done using webscraping on 3 different dealership websites. For this step **beautifulsoup** and **selenium** will be used
2. The data extracted will be merged and cleaned using **Pandas** library
3. The resulting data will be then displayed and saved in a csv file

## Install the different modules

In [ ]:
#install module
!pip install bs4
!pip install requests
!pip install webdriver-manager
!pip install selenium

In [ ]:
#import module
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

## Extract

### First source: HGregoire website
The link to the website is [hgregoire.com](https://www.hgregoire.com/).Our goal is to retrieve all the electric cars available, so we will prepare the url with the appropriate query parameters after setting them on the website:

In [ ]:
url_hgregoire = "https://www.hgregoire.com/auto-usage?price_sort=1&fuels=%C3%89lectrique%2CHybride"

Now we can get the data using beautifulsoup.

In [ ]:
# Get data to parse
data  = requests.get(url_hgregoire).text
soup = BeautifulSoup(data,"html.parser")

Then given the different attribures, we can parse the data and extract the following information: name, trim, year, mileage, price and location

In [ ]:
data_vehicle_hgreg = pd.DataFrame(columns=["Name", "Trim", "Year", "Mileage", "Price", "Location", "Link"])
vehicles = soup.find_all(attrs={"class": "car-details"})
for vehicle in vehicles:
    if vehicle.find_parent('div').find('div', attrs={"class": "srp_sold_overlay"}):
        continue
    if vehicle.find(attrs={"class": "vehicle-price"}) is None:
        continue
        
    price = vehicle.find(attrs={"class": "vehicle-price"}).find(attrs={"itemprop": "price"})['content']
    location = vehicle.find(attrs={"class": "vehicle-location"}).text.strip()
    vehicle_detail_div = vehicle.find(attrs={"class": "vehicle-detail"})
    url = vehicle_detail_div.find('a')['href']
    trim = ''
    name_span = vehicle_detail_div.find('span', attrs={"itemprop": "name"})
    trim = name_span.find_all('span', {'class': True})[0].text
    name_and_year = name_span.find_all('span', {'class': False})[0].text
    #first info is the year, and the rest is the car name
    year = name_and_year.split(' ')[0]
    name_list = name_and_year.split(' ')[1:]
    name = ' '.join(name_list)
    mileage = vehicle_detail_div.find(attrs={"itemprop": "mileageFromOdometer"}).find(attrs={"itemprop": "value"}).text
    #print(name_and_year, price , location, url, trim, mileage)
    data_vehicle_hgreg = data_vehicle_hgreg.append({"Name":name, "Trim":trim, "Year":year, "Price": price, "Mileage":mileage, "Location": location, "Link": url}, ignore_index=True)
    
data_vehicle_hgreg

### Second source: Montreal Autoprix website
The link to the website is [here](https://mtlautoprix.com/).
In order to find all the electric vehicles, we should go to the inventory and then choose the appropriate filter: 

In [ ]:
url_mtlautoprix = "https://mtlautoprix.com/inventaire-vehicules/?e=1"

Now we can get the data using beautifulsoup.

In [ ]:
data_map  = requests.get(url_mtlautoprix).text
soup_map = BeautifulSoup(data_map,"html.parser")

Then we analyze the data and extract the necessary information. On this website, we can get the name, transmission, year, mileage, and of course the price.

In [ ]:
data_vehicle_map = pd.DataFrame(columns=["Name", "Transmission", "Year", "Mileage", "Price", "Link"])
vehicles_map = soup_map.find_all(attrs={"class": "info-top-ctn"})
for vehicle in vehicles_map:
    name = vehicle.find('div',attrs={"class": "info-brand"}).find('a').text
    url = vehicle.find('div',attrs={"class": "info-brand"}).find('a')['href']
    year = vehicle.find('span', attrs={"class": "year"}).text
    price = vehicle.find('span', attrs={"class": "price-big-part"}).text + vehicle.find('span', attrs={"class": "price-small-part"}).text
    price = price[:price.find('$')]
    mileage = vehicle.find('span', attrs={"class": "mileage"}).text
    transmission = vehicle.find('span', attrs={"class": "trm"}).text
    data_vehicle_map = data_vehicle_map.append({"Name":name, "Transmission":transmission, "Year":year, "Price": price, "Mileage":mileage, "Link": url}, ignore_index=True)
    #print(name, year, price, mileage, transmission)
    
data_vehicle_map

### Third source: Automobile en direct
This is another popular car dealer website: [https://www.automobileendirect.com/](https://www.automobileendirect.com/).
In order to find all the electric vehicles, we should go to the filter available on the website and select the appropriate options: 

In [ ]:
url_aed = "https://www.automobileendirect.com/auto-usage?fuels=electric,hybrid&page=1&sortBy=date_desc"

In this case, there are 2 observations to consider:
* There is a pagination,so we should find a way to navigate through the different tabs
* When getting the page content directly with requests module and beautifulsoup like we did before, we face an issue causing the data of interest to be unavailable after a few iterations.
To address this situation, we first use selenium framework to navigate to the website and get the DOM element containing all the cars. Then we use beautifulsoup to parse this DOM element and get the car information.

In [ ]:
data_vehicle_aed = pd.DataFrame(columns=["Name", "Description", "Year", "Mileage", "Price", "Transmission", "Link"])
i=1
driver = webdriver.Chrome(ChromeDriverManager().install())
while True:
    url_aed = "https://www.automobileendirect.com/auto-usage?fuels=electric,hybrid&page="+str(i)+"&sortBy=date_desc"
    driver.get(url_aed)
    car_res= driver.find_element_by_class_name('cars__results')
    vehicles_aed = car_res.find_elements_by_class_name('carPreview')
    # check if ther is any car in the page
    if len(vehicles_aed)==0:
        break
    # parse the element with beautifulsoup
    soup_aed = BeautifulSoup(car_res.get_attribute('innerHTML'),"html.parser")
    list_vehicle = soup_aed.findChildren('a', attrs={"class": "carPreview"})
    for vehicle in list_vehicle:
        url = vehicle['href']
        name_and_year = vehicle.find(attrs={"class": "carPreview__title"}).text
        year = name_and_year.split(' ')[0]
        name_list = name_and_year.split(' ')[1:]
        name = ' '.join(name_list)
        short_desc = vehicle.find(attrs={"class": "carPreview__short-desc"}).text
        price = vehicle.find(attrs={"class": "carPreview__prices__current"}).text
        if vehicle.find(attrs={"class": "carPreview__location"}) is not None:
            transmission = vehicle.find(attrs={"class": "carPreview__location"}).text
        else:
            transmission = ''
        mileage = vehicle.find(attrs={"class": "carPreview__kms"}).text
        data_vehicle_aed = data_vehicle_aed.append({"Name":name, "Description":short_desc, "Year":year, "Price": price, "Mileage":mileage, "Transmission": transmission, "Link":url}, ignore_index=True)
    
    
    i=i+1
    
driver.close()

The columns we get are name, description, year, mileage, price and transmission

In [ ]:
data_vehicle_aed.head(10)

The Load process did not start yet, but we should add the domain name missing in the link

In [ ]:
data_vehicle_aed['Link'] = 'https://www.automobileendirect.com'+data_vehicle_aed['Link']
data_vehicle_aed.head(10)

### Merge result
Once the data are extracted, we merge them to obtain the data we need to transform.

In [ ]:
#Merge the 3 dataframes
df_concat = pd.concat([data_vehicle_hgreg, data_vehicle_map, data_vehicle_aed], ignore_index=True)

df_concat

## Transform

### Trim
Display all the values in "Trim" column.

In [ ]:
df_concat['Trim'].unique()

There are NAN values in the series. We will replace them by an empty string

In [ ]:
df_concat['Trim'] = df_concat['Trim'].fillna('')

As the trim applies only to the first set of data, we will add them to the name column and delete the trim column

In [ ]:
#Merge the 2 columns name and trim
df_concat['Name'] = (df_concat['Name'].astype(str) + ' ' + df_concat['Trim'].astype(str))

In [ ]:
df_concat.head(10)

All the words in the name column should start with an upper case letter

In [ ]:
df_concat['Name'] = df_concat['Name'].str.title()
df_concat

All electri vehicles retrieved use an automatic transmission, so the column "Transmission" is useless and it can be also deleted

In [ ]:
df_concat = df_concat.drop(columns=['Trim', 'Transmission'])
df_concat.head(10)

### Description
Display all the values in "Description" column.

In [ ]:
df_concat['Description'].unique()

At this stage we fill NaN values with "N/A" value

In [ ]:
df_concat['Description'] = df_concat['Description'].fillna('N/A')

As the description can be relevant, we will not remove the column. A further task will be to analyze all the data sets and see if we can  extract a useful and coherent description from all the web sources

### Year
For this column, we will set a different type

In [ ]:
df_concat['Year'] = df_concat['Year'].astype(int) 

### Mileage
Display all the values in "Description" column.

In [ ]:
df_concat['Mileage'].unique()

Convert the column into a string series and then perform some replacement to standardize the data: remove character, space, unit,...As we want to set a integer data type after, the "N/A" values are replaced by "-1" to identify missing values

In [ ]:
df_concat['Mileage'] = df_concat['Mileage'].astype(str)
df_concat['Mileage'] = df_concat['Mileage'].str.replace(u'\xa0', u'')
df_concat['Mileage'] = df_concat['Mileage'].str.replace(',', '')
df_concat['Mileage'] = df_concat['Mileage'].str.replace('km', '')
df_concat['Mileage'] = df_concat['Mileage'].str.replace('KM', '')
df_concat['Mileage'] = df_concat['Mileage'].str.replace('N/A', '-1')
df_concat['Mileage'] = df_concat['Mileage'].str.replace(' ', '')
df_concat['Mileage'].unique()

In [ ]:
df_concat['Mileage'] = df_concat['Mileage'].fillna('-1')
df_concat['Mileage'] = df_concat['Mileage'].astype(int)
df_concat['Mileage'].unique()

In [ ]:
# Just confirm we have the good data type
df_concat['Mileage'].dtype

### Price
Display all the values in "Price" column.

In [ ]:
df_concat['Price'].unique()

Convert the column into a string series and then perform some replacement to standardize the data

In [ ]:
df_concat['Price'] = df_concat['Price'].astype(str)
df_concat['Price'] = df_concat['Price'].str.replace(u'\xa0', u'')
df_concat['Price'] = df_concat['Price'].str.replace('$', '')
df_concat['Price'].unique()

In [ ]:
# convert into a float series
df_concat['Price'] = df_concat['Price'].astype(float)
df_concat.head(10)

### Location

In [ ]:
df_concat['Location'].unique()

For this column, we will just mark the NaN values as "to be determined", and see if the location can be extracted from all the sources 

In [ ]:
df_concat['Location'] = df_concat['Location'].fillna('TBD')
df_concat['Location'] = df_concat['Location'].astype('str')
df_concat['Location'] = df_concat['Location'].str.replace('St', 'Saint')
df_concat['Location'].unique()

## Load
The resulting dataframe is shown below

In [ ]:
df_concat

We can load results in a csv file

In [ ]:
df_concat.to_csv("electric_vehicles_etl.csv")

## Next
We complete the ETL process of obtaining the electric vehicles available to sell. Some improvements can be made to the project:
* Improve the web scraping section to obtain missing information like location and description for certain sources
* Create an endpoint to make available the data obtained from the ETL process
* Gather other useful information ( like images, color) to build a small web application and display the resuls